In [1]:
import tensorflow as tf
import cv2 
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
training_Data = []      
Datadirectory = "/Users/shashankjagtap/Desktop/Shashank before US/SEM4/sem4_pbl"
Classes = ["mask","without_mask"]
img_size = 224

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory,category)
        class_num = Classes.index(category)   
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(img_size,img_size)) 
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass

In [ ]:
create_training_Data()

In [4]:
X = []
y = []

for features,label in training_Data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1,img_size,img_size,3)
X = X/255.0

## Saving the preprocessed images in pickle

In [6]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

## Loading the images

In [ ]:
import pickle
import numpy as np

# Load X from the pickle file
with open("X.pickle", "rb") as pickle_in:
    X = pickle.load(pickle_in)

# Load y from the pickle file and convert to a NumPy array
with open("y.pickle", "rb") as pickle_in:
    y = np.array(pickle.load(pickle_in))

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import ssl

# Bypass SSL certificate verification (use with caution)
ssl._create_default_https_context = ssl._create_unverified_context

num_classes = 2
model = Sequential()
model.add(MobileNet(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3)))
model.add(Dense(num_classes, activation='softmax'))

# not using the first layer for training
model.layers[0].trainable = False

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ (None, 1024)           │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,230,914 (12.32 MB)

 Trainable params: 2,050 (8.01 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [9]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X, y, epochs = 4, validation_split = 0.25)

Epoch 1/4
44/44 ━━━━━━━━━━━━━━━━━━━━ 33s 715ms/step - accuracy: 0.6990 - loss: 0.7892 - val_accuracy: 0.8404 - val_loss: 0.3685
Epoch 2/4
44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 697ms/step - accuracy: 0.9266 - loss: 0.2224 - val_accuracy: 0.9021 - val_loss: 0.2412
Epoch 3/4
44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 701ms/step - accuracy: 0.9522 - loss: 0.1552 - val_accuracy: 0.8979 - val_loss: 0.2346
Epoch 4/4
44/44 ━━━━━━━━━━━━━━━━━━━━ 32s 742ms/step - accuracy: 0.9555 - loss: 0.1318 - val_accuracy: 0.9319 - val_loss: 0.1573


## Testing on test dataset

In [14]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Test directory containing the subfolders
test_dir = "/Users/shashankjagtap/Desktop/Shashank before US/SEM4/sem4_pbl/test"
img_size = 224  # Image size expected by the model

# Initialize counters
correct = 0
total = 0

# Map subfolder names to class labels
label_map = {
    "with_mask": 0,
    "without_mask": 1
}

def load_and_preprocess_image(img_path):
    """Load and preprocess a single image."""
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return np.expand_dims(img, axis=0)  # Add batch dimension

# Iterate through each subfolder and its images
for folder in os.listdir(test_dir):
    folder_path = os.path.join(test_dir, folder)
    
    if folder in label_map:  # Ensure it's a valid class folder
        actual_label = label_map[folder]

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            try:
                # Load and preprocess the image
                img = load_and_preprocess_image(img_path)
                prediction = model.predict(img)  # Make prediction

                # Get the predicted class (0 or 1)
                predicted_label = np.argmax(prediction, axis=1)[0]

                # Update counters
                total += 1
                if predicted_label == actual_label:
                    correct += 1

                # Print individual result
                # print(f"Image: {img_name} | Actual: {actual_label} | Predicted: {predicted_label}")

            except Exception as e:
                print(f"Error processing {img_name}: {e}")

# Calculate and print overall accuracy
accuracy = correct / total * 100 if total > 0 else 0
print(f"\nTotal Images: {total} | Correct Predictions: {correct} | Accuracy: {accuracy:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Error processing .DS_Store: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━

In [12]:
import pickle

# Save the model configuration and weights using pickle
with open('mask_model.pkl', 'wb') as file:
    pickle.dump(model, file)